In [49]:
import boto3
import json
import os
import argparse
import pyspark
from pyspark.sql import SparkSession
import tabula
import pandas as pd
from dbfread import DBF

In [11]:
# Load Credentials
with open("credentials.json", "r") as cred:
    credentials = json.load(cred)
    ACCESS_ID = credentials["accessKeyID"]
    SECRET_KEY = credentials["secretAccessKey"]

In [15]:
#Create Session
session = boto3.Session(
    aws_access_key_id = ACCESS_ID,
    aws_secret_access_key = SECRET_KEY
)

In [16]:
s3 = session.resource("s3")
bucket = s3.Bucket("land-finder-cig")

In [ ]:
def convert_dbf(file):
    dbf = DBF(file)
    df = pd.DataFrame(iter(dbf))
    
    return df

In [ ]:
def convert_excel(file):
    df = pd.read_excel(file)
    
    return df

In [68]:
def generate_schema():

    #Gather Specified widths by parsing the pdf located at link below:
    file = "https://s3.us-west-2.amazonaws.com/wakegov.com.if-us-west-2/prod/documents/2020-10/Record%20Layout.pdf"

    tables = tabula.read_pdf(file, pages=1, multiple_tables=True)
    
    #Concat tables
    tblAll = pd.concat([tables[0], tables[1]])
    
    #Generate lists to parse
    start = tblAll['Start'].to_list()
    end = (tblAll['Start'] + tblAll['Width'] - 1).to_list()
    col_name = tblAll['Field Name'].to_list()
    
    #Generate schema for Dataframe
    schema
    
    return schema

In [69]:
start, end, col_name = generate_schema()

In [70]:
#Start Spark Session
spark = SparkSession.builder.master("local[*]").appName('land-finder').getOrCreate()

In [87]:
#Load in dataset
df = spark.read.text("RealEstData06232022.txt")

In [89]:
for (st, e, fn) in zip(start, end, col_name):
    df = df.withColumn(fn, df.value.substr(st, e))

In [95]:
#df.show(1)
df.select("Owner 1", "Street Name").show()

+--------------------+--------------------+
|             Owner 1|         Street Name|
+--------------------+--------------------+
|NORTHGATE PROPERT...|WAKE FOREST      ...|
|BONES 13 LLC     ...|TRIANGLE         ...|
|BONES 13 LLC     ...|TRIANGLE         ...|
|HARRISPARK PROPER...|WAKE FOREST      ...|
|MOGHADASS, INC   ...|CAPITAL          ...|
|RALEIGH CITY OF  ...|BAILEY           ...|
|SKS PROPERTIES   ...|CHAMBERLAIN      ...|
|N C BEER WHOLESAL...|PERSON           ...|
|MICKENS, ROBERT E...|BERTIE           ...|
|COREY, JAMES CURT...|PROVIDENCE       ...|
|WOMBLE, GEORGE M ...|LAKESIDE         ...|
|MILLETTE, DAVID P...|MARSHBURN        ...|
|E&M MERCH INC    ...|JONES FRANKLIN   ...|
|ABBOTT, L FAITH  ...|BENNETT          ...|
|ABBOTT, GEORGE F ...|LAKE             ...|
|ABBOTT, HUBERT   ...|WOODLAND         ...|
|MERTZ, DERRICK & ...|DIXIE            ...|
|RALEIGH CUSTOM RE...|HARPS MILL       ...|
|CONNELL, RALPH E ...|BRANDYWINE       ...|
|DEN BLEYKER, ROBE...|BRANDYWINE

In [97]:
spark.stop()